In [1]:
import glob
from IPython.display import display
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns 
plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [2]:
path_in_train = "../data/raw/train.csv"
path_in_weather = "../data/raw/weather.csv"
path_in_test = "../data/raw/test.csv"

In [3]:
train_df = pd.read_csv(path_in_train)
wea_df = pd.read_csv(path_in_weather)
test_df = pd.read_csv(path_in_test)

In [4]:
train_df.head()

,kind,date,amount,mode_price,area
0,だいこん,20051107,201445,735.0,千葉
1,だいこん,20051108,189660,840.0,千葉_各地_青森
2,だいこん,20051110,218166,735.0,千葉_各地_青森
3,だいこん,20051111,182624,682.5,千葉_青森
4,だいこん,20051112,220691,682.5,千葉_青森


In [5]:
test_df.head()

,kind,date,area
0,だいこん,20220502,千葉_各地_青森
1,だいこん,20220506,千葉_各地_青森
2,だいこん,20220507,千葉_各地_青森
3,だいこん,20220509,千葉_各地_青森
4,だいこん,20220510,千葉_各地_青森


In [6]:
wea_df.head()

,date,mean_temp,max_temp,max_temp_time,min_temp,min_temp_time,sum_rain,sun_time,mean_humid,area
0,20041106,13.1,20.1,2004/11/6 12:50,8.1,2004/11/6 05:31,5.0,1.6,77.0,青森
1,20041107,9.6,12.5,2004/11/7 13:40,5.4,2004/11/7 22:17,0.0,4.1,63.0,青森
2,20041108,9.0,15.5,2004/11/8 12:51,3.2,2004/11/8 06:28,0.0,8.9,72.0,青森
3,20041109,12.2,17.7,2004/11/9 14:40,5.7,2004/11/9 02:49,1.5,1.2,85.0,青森
4,20041110,11.6,16.4,2004/11/10 14:42,6.9,2004/11/10 06:23,0.0,7.5,89.0,青森


In [7]:
print(train_df.shape)
print(test_df.shape)
print(wea_df.shape)

(97782, 5)
(320, 3)
(204320, 10)


# 日次データ処理

In [8]:
for df in (train_df, wea_df, test_df):
    df['date'] = pd.to_datetime(df["date"], format="%Y%m%d")
    
for i in ("max_temp_time", "min_temp_time"):
    wea_df.loc[:, i] = pd.to_datetime(wea_df[i], format="%Y/%m/%d %H:%M")

# concat

In [9]:
all_df = pd.concat([train_df,test_df])

In [10]:
all_df

,kind,date,amount,mode_price,area
0,だいこん,2005-11-07,201445.0,735.0,千葉
1,だいこん,2005-11-08,189660.0,840.0,千葉_各地_青森
2,だいこん,2005-11-10,218166.0,735.0,千葉_各地_青森
3,だいこん,2005-11-11,182624.0,682.5,千葉_青森
4,だいこん,2005-11-12,220691.0,682.5,千葉_青森
...,...,...,...,...,...
315,ミニトマト,2022-05-26,NaN,NaN,各地_愛知
316,ミニトマト,2022-05-27,NaN,NaN,各地_愛知
317,ミニトマト,2022-05-28,NaN,NaN,各地_愛知
318,ミニトマト,2022-05-30,NaN,NaN,各地_愛知


# areaを合わせる

In [11]:
all_df["area"].unique(), wea_df["area"].unique()

(array(['千葉', '千葉_各地_青森', '千葉_青森', '千葉_各地', '千葉_神奈川', '千葉_各地_神奈川', '神奈川',
        '千葉_各地_茨城', '千葉_茨城', '青森', '北海道_青森', '北海道_各地_青森', '北海道_各地',
        '北海道_千葉_各地', '各地_神奈川', '各地_青森', '各地', '北海道', '千葉_埼玉', '千葉_各地_埼玉',
        '各地_埼玉', '埼玉', '北海道_千葉', '千葉_徳島', '千葉_各地_徳島', '徳島', '各地_徳島',
        '中国_各地_徳島', '中国_北海道', '各地_徳島_静岡', '徳島_静岡', '茨城', '各地_茨城',
        '各地_茨城_青森', '茨城_青森', '佐賀_茨城', '佐賀_各地_茨城', '千葉_各地_愛知', '千葉_愛知',
        '愛知', '各地_愛知_神奈川', '岩手_群馬', '各地_岩手_群馬', '北海道_各地_群馬', '各地_群馬', '群馬',
        '千葉_各地_群馬', '千葉_群馬', '各地_愛知', '愛知_神奈川', '各地_群馬_茨城', '各地_茨城_香川',
        '各地_静岡_香川', '各地_香川', '各地_茨城_長野', '茨城_長野', '長野', '群馬_長野',
        '各地_群馬_長野', '各地_長野', '静岡_香川', '各地_茨城_静岡', '兵庫_各地_茨城', '兵庫_各地_静岡',
        '各地_静岡', '兵庫_茨城', '兵庫_各地', '兵庫_静岡', '茨城_静岡', '静岡', '各地_茨城_長崎',
        '群馬_茨城', '埼玉_東京', '各地_埼玉_東京', '各地_東京', '東京', '各地_埼玉_群馬', '埼玉_群馬',
        '各地_栃木', '栃木_群馬', '栃木', '栃木_茨城', '各地_栃木_群馬', '中国_各地_青森',
        '各地_埼玉_青森', '埼玉_青森', '各地_秋田_青森', '秋田_青森', '各地_秋田', '千葉_栃木',
        '千葉_各

## 卸売データ
各地＝全国平均、複数県＝平均値

In [12]:
# 卸売データのエリア
area_pairs = all_df["area"].unique()

yasai_areas = set()

for area_pair in area_pairs:
    areas = area_pair.split("_")
    yasai_areas = (yasai_areas | set(areas)) # 論理和
    
    
# 天候データのエリア
wea_areas = wea_df["area"].unique()

In [13]:
# マッピングのための辞書を作成
area_map = {}
update_area_map = {
    '岩手':'盛岡','宮城':'仙台','静岡':'浜松','沖縄':'那覇','神奈川':'横浜','愛知':'名古屋','茨城':'水戸','北海道':'帯広','各地':'全国',
    '兵庫':'神戸','香川':'高松','埼玉':'熊谷','国内':'全国','山梨':'甲府','栃木':'宇都宮','群馬':'前橋','愛媛':'松山'
}

for yasai_area in yasai_areas:
    if (yasai_area in wea_areas):
        area_map[yasai_area] = yasai_area
    elif (yasai_area in update_area_map):
        area_map[yasai_area] = update_area_map[yasai_area]
    else:
        area_map[yasai_area] = "全国"
        
area_map

{'静岡': '浜松',
 '栃木': '宇都宮',
 '中国': '全国',
 '埼玉': '熊谷',
 'トンガ': '全国',
 '千葉': '千葉',
 '高知': '高知',
 '香川': '高松',
 'アメリカ': '全国',
 '福島': '福島',
 '兵庫': '神戸',
 '青森': '青森',
 '愛知': '名古屋',
 '沖縄': '那覇',
 '各地': '全国',
 '福岡': '福岡',
 'メキシコ': '全国',
 '秋田': '秋田',
 '徳島': '徳島',
 '岩手': '盛岡',
 '山梨': '甲府',
 'ニュージーランド': '全国',
 '長野': '長野',
 '東京': '東京',
 '和歌山': '和歌山',
 '長崎': '長崎',
 '神奈川': '横浜',
 '熊本': '熊本',
 '山形': '山形',
 '新潟': '新潟',
 '宮崎': '宮崎',
 '鹿児島': '鹿児島',
 'カナダ': '全国',
 '北海道': '帯広',
 '佐賀': '佐賀',
 '群馬': '前橋',
 '愛媛': '松山',
 '宮城': '仙台',
 '茨城': '水戸'}

In [14]:
# 卸売データのareaを置換
all_df["area"] = all_df["area"].apply(lambda x: "_".join([area_map[i] for i in x.split("_")]))
all_df.head()

,kind,date,amount,mode_price,area
0,だいこん,2005-11-07,201445.0,735.0,千葉
1,だいこん,2005-11-08,189660.0,840.0,千葉_全国_青森
2,だいこん,2005-11-10,218166.0,735.0,千葉_全国_青森
3,だいこん,2005-11-11,182624.0,682.5,千葉_青森
4,だいこん,2005-11-12,220691.0,682.5,千葉_青森


## 天候データ
複数エリアに跨る場合はそれらの平均を取る

In [15]:
# datetime型の平均の取り方がわからないので削除
wea_df = wea_df.drop(columns=["max_temp_time","min_temp_time"])

In [16]:
area_pairs = all_df["area"].unique()
target_cols = [i for i in wea_df.columns if i not in("area","date")]
date = wea_df[wea_df["area"]=="千葉"]["date"]

In [17]:
area_pair_dfs = []

# for area_pair in area_pairs:
#     areas = area_pair.split("_")
#     # 全ての値が０のDFを作成
#     base_tmp_df = pd.DataFrame(np.zeros(wea_df[wea_df["area"]=="千葉"][target_cols].shape), columns=target_cols)
#     for area in areas:
#         tmp_df = wea_df[wea_df["area"]==area].reset_index(drop=True)[target_cols]
#         base_tmp_df = base_tmp_df.add(tmp_df)
#     base_tmp_df /= len(areas)
#     base_tmp_df["area"] = area_pair
#     base_tmp_df["date"] = date.to_list()
#     area_pair_dfs.append(base_tmp_df)
    

for area_pair in area_pairs:
    areas = area_pair.split('_')
    if len(areas) > 0:
        area = areas[0]
        base_tmp_df = wea_df[wea_df['area'] == area]
        base_tmp_df = base_tmp_df[target_cols].reset_index(drop=True)
        for area in areas[1:]:
            tmp_df = wea_df[wea_df['area'] == area]
            tmp_df = tmp_df[target_cols].reset_index(drop=True)
            base_tmp_df = base_tmp_df.add(tmp_df)
        base_tmp_df /= len(areas)
        base_tmp_df['area'] = area_pair
        area_pair_dfs.append(base_tmp_df)

In [18]:
area_pair_df = pd.concat(area_pair_dfs)
area_pair_df

,mean_temp,max_temp,min_temp,sum_rain,sun_time,mean_humid,area
0,17.40,21.20,14.10,0.00,5.10,84.0,千葉
1,17.00,22.20,13.50,0.00,7.90,77.0,千葉
2,15.80,19.00,12.70,0.00,0.60,72.0,千葉
3,16.80,21.50,12.40,0.00,7.80,75.0,千葉
4,16.70,21.90,11.50,0.00,6.70,79.0,千葉
...,...,...,...,...,...,...,...
6380,19.80,25.90,16.25,10.50,0.40,73.5,新潟_長野
6381,14.45,18.90,9.00,15.00,3.10,77.5,新潟_長野
6382,12.60,18.65,6.65,0.00,10.35,66.0,新潟_長野
6383,11.30,15.55,7.60,30.25,0.90,79.5,新潟_長野


In [19]:
area_pair_dfs[40]

,mean_temp,max_temp,min_temp,sum_rain,sun_time,mean_humid,area
0,NaN,NaN,NaN,NaN,NaN,NaN,全国_名古屋_横浜
1,NaN,NaN,NaN,NaN,NaN,NaN,全国_名古屋_横浜
2,NaN,NaN,NaN,NaN,NaN,NaN,全国_名古屋_横浜
3,NaN,NaN,NaN,NaN,NaN,NaN,全国_名古屋_横浜
4,NaN,NaN,NaN,NaN,NaN,NaN,全国_名古屋_横浜
...,...,...,...,...,...,...,...
6380,NaN,NaN,NaN,NaN,NaN,NaN,全国_名古屋_横浜
6381,NaN,NaN,NaN,NaN,NaN,NaN,全国_名古屋_横浜
6382,NaN,NaN,NaN,NaN,NaN,NaN,全国_名古屋_横浜
6383,NaN,NaN,NaN,NaN,NaN,NaN,全国_名古屋_横浜


#  結合

## all_df + wea_df
area,dateをキーとしてmerge

In [20]:
pd.merge(df_abx, df_acx, on=['a', 'x'], how='left')

NameError: name 'df_abx' is not defined